# TBX1 cohort

Variants in [TBX1](https://omim.org/entry/602054) are associated with the following diseases

- [Conotruncal anomaly face syndrome](https://omim.org/entry/217095)
- [DiGeorge syndrome](https://omim.org/entry/188400)
- [Tetralogy of Fallot](https://omim.org/entry/187500)
- [Velocardiofacial syndrome](https://omim.org/entry/192430)

We have assigned the individuals to the diagnosis that best fit the clinical description if the authors did not specifically state the disease diagnosis (for instance, [Ogata et al](https://pubmed.ncbi.nlm.nih.gov/24637876/) described the diagnosis as "22q11.2 Deletion Syndrome-Like Craniofacial Features and Hypocalcemia")

In [1]:
from pyphetools.creation import TemplateImporter
from pyphetools.visualization import IndividualTable, QcVisualizer
from IPython.display import display, HTML
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.71


In [2]:
template = "input/TBX1_individuals.xlsx"
hp_json = "../hp.json"
created_by = "0000-0002-0736-9199"

In [3]:
timporter = TemplateImporter(template=template, hp_json=hp_json, created_by=created_by)
individual_list, cvalidator = timporter.import_phenopackets_from_template();

HPO version 2024-03-06
Created encoders for 42 fields
Importing OMIM:188400, DiGeorge syndrome, HGNC:11592, TBX1,  NM_080647.1
We output 26 GA4GH phenopackets to the directory phenopackets


In [4]:
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,38


In [5]:
table = IndividualTable(cvalidator.get_error_free_individual_list())
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
II-2 (FEMALE; P51Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Abnormal facial shape (HP:0001999); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (HP:0000829); Global developmental delay (HP:0001263); Sensorineural hearing impairment (HP:0000407); excluded: Cleft palate (HP:0000175); excluded: Tetralogy of Fallot (HP:0001636); excluded: Pulmonary valve atresia (HP:0010882); excluded: Atrial septal defect (HP:0001631); excluded: Aortopulmonary collateral arteries (HP:0031834); excluded: Interrupted aortic arch type B (HP:0011613); excluded: Right aortic arch (HP:0012020); excluded: Ventricular septal defect (HP:0001629); excluded: Aplasia/Hypoplasia of the thymus (HP:0010515); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-1 (FEMALE; P26Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Abnormal facial shape (HP:0001999); Velopharyngeal insufficiency (HP:0000220); Global developmental delay (HP:0001263); excluded: Cleft palate (HP:0000175); excluded: Tetralogy of Fallot (HP:0001636); excluded: Pulmonary valve atresia (HP:0010882); excluded: Atrial septal defect (HP:0001631); excluded: Aortopulmonary collateral arteries (HP:0031834); excluded: Interrupted aortic arch type B (HP:0011613); excluded: Right aortic arch (HP:0012020); excluded: Ventricular septal defect (HP:0001629); excluded: Hypoparathyroidism (HP:0000829); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-5 (MALE; P19Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Abnormal facial shape (HP:0001999); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (HP:0000829); Global developmental delay (HP:0001263); Graves disease (HP:0100647); excluded: Cleft palate (HP:0000175); excluded: Tetralogy of Fallot (HP:0001636); excluded: Pulmonary valve atresia (HP:0010882); excluded: Atrial septal defect (HP:0001631); excluded: Aortopulmonary collateral arteries (HP:0031834); excluded: Interrupted aortic arch type B (HP:0011613); excluded: Right aortic arch (HP:0012020); excluded: Ventricular septal defect (HP:0001629); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442)
III-6 (FEMALE; P13Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Abnormal facial shape (HP:0001999); Velopharyngeal insufficiency (HP:0000220); Global developmental delay (HP:0001263); excluded: Cleft palate (HP:0000175); excluded: Tetralogy of Fallot (HP:0001636); excluded: Pulmonary valve atresia (HP:0010882); excluded: Atrial septal defect (HP:0001631); excluded: Aortopulmonary collateral arteries (HP:0031834); excluded: Interrupted aortic arch type B (HP:0011613); excluded: Right aortic arch (HP:0012020); excluded: Ventricular septal defect (HP:0001629); excluded: Hypoparathyroidism (HP:0000829); excluded: Sensorineural hearing impairment (HP:0000407); excluded: Polydactyly (HP:0010442); excluded: Graves disease (HP:0100647)
III-7 (MALE; P10Y),DiGeorge syndrome (OMIM:188400),NM_080647.1:c.1253del (heterozygous),Hypertelorism (HP:0000316); Blepharophimosis (HP:0000581); Low-set ears (HP:0000369); Narrow nose (HP:0000460); Micrognathia (HP:0000347); Abnormal facial shape (HP:0001999); Velopharyngeal insufficiency (HP:0000220); Hypoparathyroidism (